In [43]:
import pandas as pd
import numpy as np
import requests
from pycoingecko import CoinGeckoAPI
import datetime
import cufflinks as cf
import matplotlib.pyplot as plt
import altair as alt
from altair.expr import datum, if_
from datetime import date
cg = CoinGeckoAPI()

In [2]:
def change_percent24_with_suffix(df, suffix):
    prices_list = list(df['Price_{}'.format(suffix)])
    percent_change24 = []
    for i in range(len(prices_list)):
        if i == 0:
            change_24 = 0
            percent_change24.append(change_24)
        else:
            change_24 = ((prices_list[i]-prices_list[i-1])/prices_list[i-1])*100
            change_24 = round(change_24, 2)
            percent_change24.append(change_24)
    return percent_change24


In [27]:
def get_history(coin):
    coin_history = cg.get_coin_market_chart_by_id(id=coin, vs_currency='usd', days='max')

    days = [i[0] for i in coin_history['prices']]
    days = pd.to_datetime(days, unit='ms')

    prices = [i[1] for i in coin_history['prices']]
#cosmos
    days = days[:-1]
    prices = prices[:-1]

#    data = 
    df = pd.DataFrame(data={'{} Price'.format(coin.capitalize()): prices}, index=days)
    
    return df

    


In [136]:
def find_percent_diff(percent, coin):

    BTCdf = get_history('bitcoin')
    coindf = get_history(coin)

    BTCcoin = pd.concat([BTCdf, coindf], axis=1)
    #BTCcoin['Date'] = BTCcoin.index.dt.strftime("%Y-%m-%d")
    #BTCcoin.index.strftime("%Y-%m-%d")
    
    weekly_dates = pd.date_range(BTCcoin.index[0], BTCcoin.index[-1], freq='W').to_series()
    BTCcoin_weekly = BTCcoin[BTCcoin.index.isin(weekly_dates)]

    BTCcoin_weekly['BTC price percent diff'] = BTCcoin_weekly['Bitcoin Price'].pct_change() * 100
    BTCcoin_weekly['{} price percent diff'.format(coin.capitalize())] = BTCcoin_weekly['{} Price'.format(coin.capitalize())].pct_change() * 100
    
    dropdf = BTCcoin_weekly[BTCcoin_weekly['BTC price percent diff'] <= -1 * percent]
    dropdf['{} dropped more'.format(coin.capitalize())] = dropdf['BTC price percent diff'] > dropdf['{} price percent diff'.format(coin.capitalize())]
    
    
    ################################# Chart ########################
    dropdf_chart = alt.Chart(dropdf.reset_index(), title="When Bitcoin drops {}%, what does {} do?".format(percent, coin.capitalize())).mark_bar().encode(
                alt.X('yearmonthdate(index):N', title="Date"),
                #y='ETH 1W change percent',
                alt.Y('{} price percent diff'.format(coin.capitalize()), axis=alt.Axis(title="{} price drop (%)".format(coin.capitalize()))),
                tooltip=['BTC price percent diff', '{} price percent diff'.format(coin.capitalize())],
                color=alt.Color('{} dropped more'.format(coin.capitalize()), scale=alt.Scale(scheme='paired'), legend=alt.Legend(title="Drop compared to BTC"))).transform_calculate(
                    '{} dropped more'.format(coin.capitalize()), if_(datum['{} dropped more'.format(coin.capitalize())] == False, 'Less', 'More')).interactive()


    return dropdf, dropdf_chart


In [152]:
BTCETHdf, BEchart = find_percent_diff(20, 'ethereum')
BTCETHdf.to_csv('BTCETH20drop_V2.csv')



In [155]:
BTCETHdf

,Bitcoin Price,Ethereum Price,BTC price percent diff,Ethereum price percent diff,Ethereum dropped more
2013-07-07,68.083100,NaN,-28.625538,NaN,False
2013-12-08,743.110000,NaN,-28.090072,NaN,False
2013-12-22,650.270000,NaN,-23.911212,NaN,False
2015-01-18,208.989200,NaN,-21.780795,NaN,False
2017-07-16,1927.099066,156.468992,-23.504058,-34.464435,True
2017-11-12,5866.513129,314.166128,-20.896694,5.595911,False
2017-12-24,14451.114547,686.208309,-25.600150,-5.831225,False
2018-02-04,8994.914866,941.803230,-22.894321,-23.883874,True
2018-03-11,8890.865804,684.934340,-21.708055,-18.871178,False
2018-11-25,3895.836813,114.287931,-30.421441,-34.940734,True


In [153]:

BTClinkdf, BLchart = find_percent_diff(20, 'chainlink')
BTClinkdf.to_csv('BTClink20drop_V2.csv')

In [148]:
#pd.read_csv('BTCETH10drop_V2.csv', index_col=0)

In [154]:

BTCADAdf, BAchart = find_percent_diff(20, 'cardano')
BTCADAdf.to_csv('BTCADA20drop_V2.csv')


In [ ]:
BAchart

In [141]:
BTCETHdf, linn = find_percent_diff(10, 'chainlink')
linn

alt.Chart(...)

In [53]:
BTCETH = alt.Chart(BTCETHdf.reset_index(), title="When Bitcoin drops 20%, what does Ethereum do?").mark_bar().encode(
                alt.X('index'),
                #y='ETH 1W change percent',
                alt.Y('ETH price percent diff', axis=alt.Axis(title="ETH price drop (%)")),
                tooltip=['BTC price percent diff', 'ETH price percent diff'],
                color=alt.Color('ETH dropped more', scale=alt.Scale(scheme='paired'), legend=alt.Legend(title="Drop compared to BTC"))).transform_calculate(
                    'ETH dropped more', if_(datum['ETH dropped more'] == False, 'Less', 'More')).interactive()
BTCETH

alt.Chart(...)

In [52]:
BTCETHW.tail()

,Bitcoin Price,Ethereum Price,BTC price percent diff,ETH price percent diff
2021-02-28,46653.526399,1480.129577,-17.248165,-23.284215
2021-03-07,49019.368592,1661.927970,5.071090,12.282600
2021-03-14,61315.197933,1927.720720,25.083614,15.993037
2021-03-21,58376.158471,1817.860143,-4.793329,-5.698988
2021-03-28,55832.418110,1713.837876,-4.357499,-5.722237


In [ ]:
BTC = get_history('bitcoin')
ETH = get_history('ethereum')

BTCETH = BTC.join(ETH, lsuffix='_BTC', rsuffix='_ETH', how='inner')
weekly_dates = pd.date_range(BTCETH.index[0], BTCETH.index[-1], freq='W').to_series()
BTCETHW = BTCETH[BTCETH.index.isin(weekly_dates)]

change_24_BTC = change_percent24_with_suffix(BTCETHW, 'BTC')
BTCETHW['BTC 1W change percent'] = change_24_BTC

change_24_ETH = change_percent24_with_suffix(BTCETHW, 'ETH')
BTCETHW['ETH 1W change percent'] = change_24_ETH

BTCETHW10 = BTCETHW[BTCETHW['BTC 1W change percent'] <= -20 ]

BTCETHW10['ETH dropped more'] = BTCETHW10['BTC 1W change percent'] > BTCETHW10['ETH 1W change percent']

BTCETHW10.to_csv('BTCETH20drop.csv', index_label='Date')

In [ ]:

BTCdf = get_history('bitcoin')
coindf = get_history('ethereum')

In [ ]:
def find_percent_diff(percent, coin):

    BTCdf = get_history('bitcoin')
    coindf = get_history(coin)

    BTCcoin = pd.concat([BTCdf, coindf], axis=1)
    #BTCcoin['Date'] = BTCcoin.index.dt.strftime("%Y-%m-%d")
    #BTCcoin.index.strftime("%Y-%m-%d")
    
    weekly_dates = pd.date_range(BTCcoin.index[0], BTCcoin.index[-1], freq='W').to_series()
    BTCcoin_weekly = BTCcoin[BTCcoin.index.isin(weekly_dates)]

    BTCcoin_weekly['BTC price percent diff'] = BTCcoin_weekly['Bitcoin Price'].pct_change() * 100
    BTCcoin_weekly['{} price percent diff'.format(coin.capitalize())] = BTCcoin_weekly['{} Price'.format(coin.capitalize())].pct_change() * 100
    
    dropdf = BTCcoin_weekly[BTCcoin_weekly['BTC price percent diff'] <= -1 * percent]
    dropdf['{} dropped more'.format(coin.capitalize())] = dropdf['BTC price percent diff'] > dropdf['{} price percent diff'.format(coin.capitalize())]
    
    
    ################################# Chart ########################
    dropdf_chart = alt.Chart(dropdf.reset_index(), title="When Bitcoin drops {}%, what does {} do?".format(percent, coin.capitalize())).mark_bar().encode(
                alt.X('yearmonthdate(index):N', title="Date"),
                #y='ETH 1W change percent',
                alt.Y('{} price percent diff'.format(coin.capitalize()), axis=alt.Axis(title="{} price drop (%)".format(coin.capitalize()))),
                tooltip=['BTC price percent diff', '{} price percent diff'.format(coin.capitalize())],
                color=alt.Color('{} dropped more'.format(coin.capitalize()), scale=alt.Scale(scheme='paired'), legend=alt.Legend(title="Drop compared to BTC"))).transform_calculate(
                    '{} dropped more'.format(coin.capitalize()), if_(datum['{} dropped more'.format(coin.capitalize())] == False, 'Less', 'More')).interactive()


    return dropdf, dropdf_chart


In [ ]:
def get_history(coin):
    coin_history = cg.get_coin_market_chart_by_id(id=coin, vs_currency='usd', days='max')

    days = [i[0] for i in coin_history['prices']]
    days = pd.to_datetime(days, unit='ms')

    prices = [i[1] for i in coin_history['prices']]
#cosmos
    days = days[:-1]
    prices = prices[:-1]

#    data = 
    df = pd.DataFrame(data={'{} Price'.format(coin.capitalize()): prices}, index=days)
    
    return df

    
